In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pylab

import seaborn as sns

import csv

import scipy
from scipy import stats

In [ ]:
%matplotlib inline
import deepdish as dd
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as patches
import numpy as np
import brainiak.eventseg.event
from scipy import stats
from scipy.stats import norm, zscore, pearsonr
from scipy.signal import gaussian, convolve
from sklearn import decomposition
import seaborn as sns 

In [ ]:
#import cupy as cp
from easyesn.optimizers import GradientOptimizer
from easyesn import PredictionESN
from easyesn.optimizers import GridSearchOptimizer
from easyesn import helper as hlp
import numpy as np
#from wikipedia2vec import Wikipedia2Vec

#import nltk
#nltk.download('stopwords')
#from nltk.corpus import stopwords


vectorDim = 100

numNode = 1000

reps = 100

# the data structures to save the means for the coherent and incoherent
coherent_means = np.empty(reps)
incoherent_means = np.empty(reps)

coherent_vars_1 = np.empty(reps)
coherent_vars_2 = np.empty(reps) 

incoherent_vars_1 = np.empty(reps)
incoherent_vars_2 = np.empty(reps) 



#here is a small bit of data to train the reservoir.  We dont care
# if it learns because we are analysing the reseroir units, and
#not the readouts

in_small = np.load('in_small.npy')
out_small = np.load('out_small.npy')

In [ ]:
#reservoirStatesBuffer[:,:resn]


print(len(in_small))
print(len(out_small))

In [ ]:
# in a different program we saved these data
#np.save('inputDataTestingShift.npy', inputDataTestingShift)
#np.save('inputDataTestingIntact.npy', inputDataTestingIntact)


In [ ]:
#here we import the word vectors for the intact and shifted 
# not the fall texts.  Avoids using the big wikipedia2vec pkl file

inputDataTestingIntact = np.load('inputDataTestingIntact.npy')
inputDataTestingShift = np.load('inputDataTestingShift.npy')


In [ ]:
# now the radical loop: 1. run 2 reservoirs; 2. calculate time contst; 3.sort;  4. segment (ubuntu)

for res_seed in range(reps):

    ###################################################################
    #
    #               1.1 run reservoir on intact text


    # set the seed
    np.random.seed(res_seed)

    # set up the reservoir
    # leakingRate=0.2 changing too fast, mutliple with 0.05; changed
    esn = PredictionESN(n_input=vectorDim, n_output=vectorDim, n_reservoir=numNode, leakingRate=0.05, regressionParameters=[1e-2], solver="lsqr", feedback=False)

    # train the reervoir with a new seed
    esn.fit(in_small, out_small, transientTime="Auto", verbose=1)

    #run the reservoir
    reservoirStatesBuffer = np.empty((0,numNode))
    prediction,reservoirStatesBuffer  = esn.predict(inputDataTestingIntact)
    reservoirStatesBuffer = reservoirStatesBuffer.T
    print(reservoirStatesBuffer.shape)
    reservoirStatesBuffer = reservoirStatesBuffer[:,101:]
    print(reservoirStatesBuffer.shape)
    print(prediction.shape)

    reservoirStatesBuffer1 = reservoirStatesBuffer

    ###################################################################
    #
    #               1.2 run reservoir on shifted text

    # set the seed
    np.random.seed(res_seed)

    # set up the reservoir
    # leakingRate=0.2 changing too fast, mutliple with 0.05; changed
    esn = PredictionESN(n_input=vectorDim, n_output=vectorDim, n_reservoir=numNode, leakingRate=0.05, regressionParameters=[1e-2], solver="lsqr", feedback=False)

    # train the reervoir with a new seed
    esn.fit(in_small, out_small, transientTime="Auto", verbose=1)

    #run the reservoir
    reservoirStatesBuffer = np.empty((0,numNode))
    prediction,reservoirStatesBuffer  = esn.predict(inputDataTestingShift)
    reservoirStatesBuffer = reservoirStatesBuffer.T
    print(reservoirStatesBuffer.shape)
    reservoirStatesBuffer = reservoirStatesBuffer[:,101:]
    print(reservoirStatesBuffer.shape)
    print(prediction.shape)

    reservoirStatesBuffer2 = reservoirStatesBuffer

    resn = 50
    begin=0
    end=680
    fig= plt.figure(figsize=(16,4))

    plt.plot( reservoirStatesBuffer1[begin:end,:resn] - reservoirStatesBuffer2[begin:end,:resn])
    plt.title('Reservoir Temporal Structure Intact vs Shifted')
    plt.axvline(x=139, color='b', linestyle='--')
    plt.axvline(x=433, color='b', linestyle='--')


    fig.tight_layout()
    plt.savefig('./CH2020-results/'+str(res_seed)+'.png')



    ###################################################################
    #
    #               2. calculate time contst

    # get the subset of data that has the different-to-same shift

    resn = numNode
    index = 0
    begin=433
    end=683
    data = reservoirStatesBuffer1[begin:end,index:index+resn] - reservoirStatesBuffer2[begin:end,index:index+resn]
    print(data.shape)
    dataT=data.T

    timeconstants = np.zeros(dataT.shape[0])
    res_maxes = np.zeros(dataT.shape[0])

    #calculate the alignment time:  dataT(neurons,time)
    #interate over reservoir neuron
    for neuron in range(dataT.shape[0]):
        # get the value
        max = abs(dataT[neuron,0])
        if max > -1:  #WAS 0.05
            for timestep in range(dataT.shape[1]):
                if abs(dataT[neuron,timestep]) < max/2:
                    timeconstants[neuron] = timestep
                    res_maxes[neuron] = max
                    max=-1     
                    
    ###################################################################
    #
    #               3.  sort
    
    save_timeconstants = timeconstants
    timeconstants = np.sort(save_timeconstants)
    # these sorted time constants can be used to display the nice distribution
    
    #get the indices - that is (timesteps, neuron)
    timesteps = reservoirStatesBuffer1.shape[0]
    neurons = reservoirStatesBuffer1.shape[1]
    
    # add a column to store the time constants into the 

    reservoirStates1SortedT = np.zeros((neurons,timesteps+1))
    reservoirStates1SortedT[:,:-1] = reservoirStatesBuffer1.T

    #create a new column for the time constants
    reservoirStates1SortedT[:,682]=save_timeconstants
    
    #do the sort, then remove the column of time constants
    SortedStates = reservoirStates1SortedT[np.argsort(reservoirStates1SortedT[:, 682])]
    temp=SortedStates[:,0:682]
    NTF_SortedStates = temp.T
    
    
    ###################################################################
    #
    #               4. Segment

    # get the pieces
    NTF_time1=NTF_SortedStates[:,100:199]
    NTF_time2=NTF_SortedStates[:,800:899]
    
    NTF_time1_cumul = NTF_time1
    NTF_time2_cumul = NTF_time2
    

    NTF_time1a=NTF_SortedStates[:,0:99]
    NTF_time1b=NTF_SortedStates[:,200:299]
    NTF_time1c=NTF_SortedStates[:,300:399]
    NTF_time1d=NTF_SortedStates[:,400:499]


    NTF_time2a=NTF_SortedStates[:,500:599]
    NTF_time2b=NTF_SortedStates[:,600:699]
    NTF_time2c=NTF_SortedStates[:,700:799]
    NTF_time2d=NTF_SortedStates[:,900:999]
    
    #simple data structure for testing
    coherent = np.zeros(8)
    incoherent = np.zeros(8)
    
    # Train the two HMS with the predicted segments
    ev_time1 = brainiak.eventseg.event.EventSegment(24)
    ev_time1.fit(NTF_time1)

    ev_time2 = brainiak.eventseg.event.EventSegment(8)
    ev_time2.fit(NTF_time2)
    
    # save the variablility scores
    
    coherent_ev1_var = ev_time1.event_var_
    coherent_ev2_var = ev_time2.event_var_

    #now test transfer
    print("Coherent training")
    a,coherent[0] = ev_time1.find_events(NTF_time1a,scramble=False)
    a,coherent[1] = ev_time1.find_events(NTF_time1b,scramble=False)
    a,coherent[2] = ev_time1.find_events(NTF_time1c,scramble=False)
    a,coherent[3] = ev_time1.find_events(NTF_time1d,scramble=False)

    a,coherent[4] = ev_time2.find_events(NTF_time2a,scramble=False)
    a,coherent[5] = ev_time2.find_events(NTF_time2b,scramble=False)
    a,coherent[6] = ev_time2.find_events(NTF_time2c,scramble=False)
    a,coherent[7] = ev_time2.find_events(NTF_time2d,scramble=False)
    print("Coheret")
    print(*coherent, sep='\n')



    # 3 display the segmentation
    color='jet'
    ec='black'
    scale=0.9
    lw=3

    nTR=683
    fig=plt.figure(figsize=(16, 8))

    fig.add_subplot(121)
    bounds = np.where(np.diff(np.argmax(ev_time1.segments_[0], axis=1)))[0]

    plt.imshow(np.corrcoef(NTF_time1),cmap=color, vmin=scale, vmax=1)

    ax = plt.gca()
    bounds_aug = np.concatenate(([0],bounds,[nTR]))
    for i in range(len(bounds_aug)-1):
        rect = patches.Rectangle((bounds_aug[i],bounds_aug[i]),bounds_aug[i+1]-bounds_aug[i],bounds_aug[i+1]-bounds_aug[i],linewidth=lw, linestyle='--',edgecolor=ec,facecolor='none')
        ax.add_patch(rect)

    plt.xlabel('Reservoir TRs')
    plt.ylabel('Reservoir TRs')

    fig.add_subplot(122)
    bounds = np.where(np.diff(np.argmax(ev_time2.segments_[0], axis=1)))[0]

    plt.imshow(np.corrcoef(NTF_time2),cmap=color, vmin=scale, vmax=1)
    ax = plt.gca()
    bounds_aug = np.concatenate(([0],bounds,[nTR]))
    for i in range(len(bounds_aug)-1):
        rect = patches.Rectangle((bounds_aug[i],bounds_aug[i]),bounds_aug[i+1]-bounds_aug[i],bounds_aug[i+1]-bounds_aug[i],linewidth=lw, linestyle='--',edgecolor=ec,facecolor='none')
        ax.add_patch(rect)

    plt.xlabel('Reservoir TRs')
    plt.ylabel('Reservoir TRs')

    fig.tight_layout()
    plt.savefig('hmm_notthefall_reservoir1.png')
    
    fig.tight_layout()
    plt.savefig('./CH2020-results/CH2020-'+str(res_seed)+'.png')

    # Train the two HMS with the NON predicted segments
    ev_time1 = brainiak.eventseg.event.EventSegment(8)
    ev_time1.fit(NTF_time1)

    ev_time2 = brainiak.eventseg.event.EventSegment(24)
    ev_time2.fit(NTF_time2)
    
    # save the variablility scores
    
    incoherent_ev1_var = ev_time1.event_var_
    incoherent_ev2_var = ev_time2.event_var_


    print("In-Coherent training")
    a,incoherent[4] = ev_time1.find_events(NTF_time1a,scramble=False)
    a,incoherent[5] = ev_time1.find_events(NTF_time1b,scramble=False)
    a,incoherent[6] = ev_time1.find_events(NTF_time1c,scramble=False)
    a,incoherent[7] = ev_time1.find_events(NTF_time1d,scramble=False)

    a,incoherent[0] = ev_time2.find_events(NTF_time2a,scramble=False)
    a,incoherent[1] = ev_time2.find_events(NTF_time2b,scramble=False)
    a,incoherent[2] = ev_time2.find_events(NTF_time2c,scramble=False)
    a,incoherent[3] = ev_time2.find_events(NTF_time2d,scramble=False)
    print("InCoheret")
    print(*incoherent, sep='\n')
    
    #consolidat the means
    coherent_means[res_seed]   = np.mean(coherent)
    incoherent_means[res_seed] = np.mean(incoherent)
    
    # consolidate the variablility scores
    
    coherent_vars_1[res_seed] = coherent_ev1_var 
    coherent_vars_2[res_seed] = coherent_ev2_var 

    incoherent_vars_1[res_seed] = incoherent_ev1_var 
    incoherent_vars_2[res_seed] = incoherent_ev2_var 
    
    # accumulate the activations
    NTF_time1_cumul=NTF_time1+NTF_time1_cumul
    NTF_time2_cumul=NTF_time2+NTF_time2_cumul


In [ ]:
NFT_time1 = NTF_time1_cumul/reps
NFT_time2 = NTF_time2_cumul/reps

#save the data
np.save('NTF-time1.npy', NFT_time1) 
np.save('NTF-time2.npy', NFT_time2) 

# Train the two HMS with the predicted segments
ev_time1 = brainiak.eventseg.event.EventSegment(24)
ev_time1.fit(NTF_time1)
            
ev_time2 = brainiak.eventseg.event.EventSegment(8)
ev_time2.fit(NTF_time2)

In [ ]:
# 3 display the segmentation
color='jet'
ec='black'
scale=0.9
lw=3

nTR=682
fig=plt.figure(figsize=(16, 8))

fig.add_subplot(121)
bounds = np.where(np.diff(np.argmax(ev_time1.segments_[0], axis=1)))[0]

plt.imshow(np.corrcoef(NTF_time1),cmap=color, vmin=scale, vmax=1)

ax = plt.gca()
bounds_aug = np.concatenate(([0],bounds,[nTR]))
for i in range(len(bounds_aug)-1):
    rect = patches.Rectangle((bounds_aug[i],bounds_aug[i]),bounds_aug[i+1]-bounds_aug[i],bounds_aug[i+1]-bounds_aug[i],linewidth=lw, linestyle='--',edgecolor=ec,facecolor='none')
    ax.add_patch(rect)

plt.xlabel('Reservoir TRs')
plt.ylabel('Reservoir TRs')

fig.add_subplot(122)
bounds = np.where(np.diff(np.argmax(ev_time2.segments_[0], axis=1)))[0]

plt.imshow(np.corrcoef(NTF_time2),cmap=color, vmin=scale, vmax=1)
ax = plt.gca()
bounds_aug = np.concatenate(([0],bounds,[nTR]))
for i in range(len(bounds_aug)-1):
    rect = patches.Rectangle((bounds_aug[i],bounds_aug[i]),bounds_aug[i+1]-bounds_aug[i],bounds_aug[i+1]-bounds_aug[i],linewidth=lw, linestyle='--',edgecolor=ec,facecolor='none')
    ax.add_patch(rect)

plt.xlabel('Reservoir TRs')
plt.ylabel('Reservoir TRs')

fig.tight_layout()
plt.savefig('hmm_notthefall_reservoir_cumul-50bis.png')

In [ ]:
#data_to_plot = (test)
fig=plt.figure(figsize=(4, 4))
#f,ax = plt.subplots(1,1, figsize=(5, 5))
plt.boxplot(data_to_plot,showmeans=True)
#plt.xlabel('Coherent                                    Incoherent')
plt.ylabel('Model Fit')
plt.axhline(y=0, color='r', linestyle='--')

plt.xticks([1], ['Temporal Segmentation Effect'])#ax.set_xticklabels(['Coherent-Incoherent'])
plt.title('N=100, pvalue=4.03e-06')
fig.tight_layout()
plt.savefig('hmm_NTF-time-scales-coherent_incoherent-difference-100.png')

In [ ]:
print(stats.ttest_rel(coherent_means,incoherent_means))
print(stats.describe(coherent_means))
print(stats.describe(incoherent_means))

print(coherent_means-incoherent_means)

In [ ]:
print(data_to_plot)
np.save('NTF-data_to_plot.npy', data_to_plot) 

In [ ]:
print(res_seed)